In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

df_main = pd.read_csv('../input/crop-yield-data-bangladesh/Crop yield data.csv')
df_main.info()

df = df_main
df = df.drop(df.columns[[8, 21, 22, 23]], axis = 1, inplace = False)
df.info()

/kaggle/input/crop-yield-data-bangladesh/Crop yield data.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 43 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   District                                                                                 384 non-null    object 
 1   Year                                                                                     384 non-null    object 
 2   Crop Yield Aus(M. Ton/Hectare)                                                           377 non-null    float64
 3   Crop Yield Aman(M. Ton/Hectare)                                                          384 non-null    float64
 4   Crop Yield Boro(M. Ton/Hectare)                                                          384 non-null    float64
 5   Cro

In [2]:
df = df.drop(df.columns[[2,3,5,6,7,8,9,11,12,14,15,17,18]], axis = 1, inplace = False)
df['Relative Humidity Winter'] = df['Relative Humidity Winter'].fillna(df['Relative Humidity Winter'].mean())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384 entries, 0 to 383
Data columns (total 26 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   District                                                                                 384 non-null    object 
 1   Year                                                                                     384 non-null    object 
 2   Crop Yield Boro(M. Ton/Hectare)                                                          384 non-null    float64
 3   Relative Humidity Winter                                                                 384 non-null    float64
 4   Total Rainfall in Winter(In millimeter)                                                  384 non-null    int64  
 5   Monthly Average Min Temp. Winter(In Celsius)                    

In [3]:
from scipy.stats import zscore

# df['zscore_yield'] = zscore(df['Crop Yield Boro(M. Ton/Hectare)'])
# df[(df.zscore_yield > 2.2)|(df.zscore_yield < -2.2)]
# df = df.drop(df[(df.zscore_yield > 2.2)|(df.zscore_yield < -2.2)].index)

df['zscore_mx_temp'] = zscore(df['Monthly Average Max Temp. Winter(In Celsius)'])
df[(df.zscore_mx_temp > 2)|(df.zscore_mx_temp < -2)]
df = df.drop(df[(df.zscore_mx_temp > 2)|(df.zscore_mx_temp < -2)].index)

df['zscore_hm'] = zscore(df['Relative Humidity Winter'])
df[(df.zscore_hm > 2.5)|(df.zscore_hm < -2.5)]
df = df.drop(df[(df.zscore_hm > 2.5)|(df.zscore_hm < -2.5)].index)

df['zscore_mn_temp'] = zscore(df['Monthly Average Min Temp. Winter(In Celsius)'])
df[(df.zscore_mn_temp > 2.5)|(df.zscore_mn_temp < -2.5)]
df = df.drop(df[(df.zscore_mn_temp > 2.5)|(df.zscore_mn_temp < -2.5)].index)

# df['zscore_rain'] = zscore(df['Total Rainfall in Winter(In millimeter)'])
# df[(df.zscore_rain > 3)|(df.zscore_rain < -3)]
# df = df.drop(df[(df.zscore_rain > 3)|(df.zscore_rain < -3)].index)

# df = df.drop(['zscore_mx_temp', 'zscore_mn_temp', 'zscore_rain', 'zscore_hm'],axis=1)
df = df.drop(['zscore_mx_temp', 'zscore_hm', 'zscore_mn_temp'],axis=1)
df.shape

(345, 26)

In [4]:
df_x = df
df_x = df_x.drop(df_x.columns[[2]], axis = 1, inplace = False)
df_x.info()

df_x = pd.get_dummies(df_x, prefix=['District', 'Year'], columns=['District', 'Year'], drop_first = True)
df_x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 345 entries, 0 to 383
Data columns (total 25 columns):
 #   Column                                                                                   Non-Null Count  Dtype  
---  ------                                                                                   --------------  -----  
 0   District                                                                                 345 non-null    object 
 1   Year                                                                                     345 non-null    object 
 2   Relative Humidity Winter                                                                 345 non-null    float64
 3   Total Rainfall in Winter(In millimeter)                                                  345 non-null    int64  
 4   Monthly Average Min Temp. Winter(In Celsius)                                             345 non-null    float64
 5   Monthly Average Max Temp. Winter(In Celsius)                    

In [5]:
# import seaborn as sn
# import matplotlib.pyplot as plt

# plt.figure(figsize = (100, 80))
# sn.heatmap(df_x.corr(), annot = True)

In [6]:
# df_x = df_x.drop('Year_2016-17', axis = 1)

# plt.figure(figsize = (100, 80))
# sn.heatmap(df_x.corr(), annot = True)

In [7]:
# df_x = df_x.drop('Total Rainfall in Summer(In millimeter)', axis = 1)

In [8]:
x = df_x.iloc[:,:].values
y = df.iloc[:,2].values

In [9]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 0)

from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.metrics import r2_score
import numpy as np

regressor = RandomForestRegressor(n_estimators = 500, random_state = 0, criterion = 'mae', max_features = 'auto')

y_pred = regressor.fit(x_train, y_train).predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred) * 100)
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred) * 100)
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)) * 100)
print('R2 Score:', r2_score(y_test, y_pred) * 100)

Mean Absolute Error: 18.700178260869485
Mean Squared Error: 5.90134579436953
Root Mean Squared Error: 24.292685718893928
R2 Score: 78.68062029972454


In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 0)

from sklearn.svm import SVR

svr = SVR(kernel = 'linear', C = 0.7, epsilon = 0)

y_pred = svr.fit(x_train, y_train).predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred) * 100)
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred) * 100)
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)) * 100)
print('R2 Score:', r2_score(y_test, y_pred) * 100)

Mean Absolute Error: 12.85067657853309
Mean Squared Error: 2.790485255608215
Root Mean Squared Error: 16.704745600003058
R2 Score: 89.91900885233798


In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = .2, random_state = 0)

from sklearn.ensemble import VotingRegressor

r1 = RandomForestRegressor(n_estimators = 500, random_state = 0, criterion = 'mae', max_features = 'auto')
r2 = SVR(kernel = 'linear', C = .7, epsilon = 0)

er = VotingRegressor([('rf', r1), ('svr', r2)])

y_pred = er.fit(x_train, y_train).predict(x_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred) * 100)
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred) * 100)
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)) * 100)
print('R2 Score:', r2_score(y_test, y_pred) * 100)

Mean Absolute Error: 14.533026596685925
Mean Squared Error: 3.6058710852288156
Root Mean Squared Error: 18.989131326179233
R2 Score: 86.97332142617643
